Import necessary packages and open up freshly cleaned dataset

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search
import ipyleaflet

In [ ]:
wind_turbines = gpd.read_file('../DataPrep/wind_turbines.shp')

In [ ]:
# Make a folium map
m = wind_turbines.explore(
    column='Developmen',
    marker_type='marker',
    popup=True,
    legend=False,
    tiles='OpenStreetMap')

m # show map

In [ ]:
# modify pop-up settings to only show relevant information, i.e. if N/A, dont show
# but only for columns related to planning process i.e. from planning application withdrawn column to operational column (last column)

In [ ]:
# add more base map options
# add additional raster data for viewing context
# change markers to show different colour by development status and by height category. 
# e.g. different colour by development status, different shape or size by height 
# perhaps define a function for this so viewer can change markers to reflect two categories like this

In [ ]:
# add search function to map

In [ ]:
# look for a way to configure a link to the planning application from the data provided 
# e.g. taking into account the planning application reference number and the planning authority